https://www.kaggle.com/code/liangxh64/exercise-missing-values/edit

In [1]:
import numpy as np
import pandas as pd

In [2]:
import mlbox
from mlbox.common.data_loader import DataLoader

data_loader = DataLoader()

In [3]:
X, y, X_test = data_loader.load_home_data()

In [4]:
X = X.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [6]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [7]:
y_train.head()

Id
619    314813
871    109500
93     163500
818    271000
303    205000
Name: SalePrice, dtype: int64

In [8]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [10]:
# CASE 1
# 去除带有空值的列
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


In [11]:
# CASE 2
from sklearn.impute import SimpleImputer

# 补均值
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE (Drop columns with missing values):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
18062.894611872147


In [12]:
cols_with_missing

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [22]:
X_train[cols_with_missing].head()

,LotFrontage,MasVnrArea,GarageYrBlt
Id,,,
619,90.0,452.0,2007.0
871,60.0,0.0,1962.0
93,80.0,0.0,1921.0
818,NaN,148.0,2002.0
303,118.0,150.0,2001.0


In [26]:
# CASE 2.1
new_X_train = X_train.copy()
new_X_valid = X_valid.copy()

# 补零
cols = ['LotFrontage', 'MasVnrArea']
new_X_train[cols] = new_X_train[cols].fillna(0)
new_X_valid[cols] = new_X_valid[cols].fillna(0)

col = 'GarageYrBlt'
default_value = X_train[col].min()
new_X_train[col] = new_X_train[col].fillna(default_value)
new_X_valid[col] = new_X_valid[col].fillna(default_value)

# 略好于补均值
print("MAE (Drop columns with missing values):")
print(score_dataset(new_X_train, new_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
18001.326929223742


In [28]:
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

print("MAE (Drop columns with missing values):")
print(score_dataset(final_X_train, final_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17791.59899543379


In [30]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.fit_transform(X_test))
final_X_test.columns = X_test.columns


model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

preds_test = model.predict(final_X_test)

/Users/lxh/.virtualenv/alisa/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [31]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)